In [2]:
### Dataset Index exchange
from pathlib import Path 
scenes = ["spagna_train0", "ciampino_train0", "ciampino_train1", "campus_train0", "campus_train1"]
start_idx = [0,0,30985,0,12048]


IMG_ROOT="/datasets/vbr_slam"
DEPTH_ROOT="/home/bjangley/VPR/vbr/depths"
POSES_ROOT="/home/bjangley/VPR/vbr/poses" 

def get_paths_from_scene(scene, img_root, depth_root, pose_root):
    depth_dir = Path(depth_root)/f"{scene}"
    scene_name = scene.split("_")[0]
    img_dir = Path(img_root)/f"{scene_name}"/f"{scene}_kitti"
    pose_dir = Path(pose_root)/f"{scene}"
    calib_path = Path (img_root)/f"{scene_name}"/f"{scene}"
    return depth_dir, img_dir, pose_dir, calib_path

print(get_paths_from_scene(scenes[0],img_root=IMG_ROOT, depth_root=DEPTH_ROOT, pose_root=POSES_ROOT))

(PosixPath('/home/bjangley/VPR/vbr/depths/spagna_train0'), PosixPath('/datasets/vbr_slam/spagna/spagna_train0_kitti'), PosixPath('/home/bjangley/VPR/vbr/poses/spagna_train0'), PosixPath('/datasets/vbr_slam/spagna/spagna_train0'))


In [ ]:
# import pandas as pd
# import os
# scene="ciampino"
# # Input file and parameters
# pairs_file = "/home/bjangley/VPR/mast3r-v2/pairs/pairs_ciampinov1/ciampino_matches_inliers_fm_top3_anchors_per_query.csv"
# pairs_df = pd.read_csv(pairs_file)

# # Define the ranges for campus_train0 and campus_train1
# train0_range = [0, 30985]
# train1_range = [30985, 49831]


# # Create output directories if they don't exist
# output_dir = "/home/bjangley/VPR/mast3r-v2/pairs/"
# os.makedirs(output_dir, exist_ok=True)

# # Output file paths
# train0_output = os.path.join(output_dir, f"{scene}_train0_pairs.csv")
# train1_output = os.path.join(output_dir, f"{scene}_train1_pairs.csv")

# # Function to filter pairs based on anchor index range and adjust indices
# def filter_pairs_by_range(df, anchor_range, index_offset=0):
#     filtered_df = df[(df['anchor_idx'] >= anchor_range[0]) & (df['anchor_idx'] < anchor_range[1])].copy()
#     # Adjust indices by subtracting the offset
#     filtered_df.loc[:, 'anchor_idx'] = filtered_df['anchor_idx'] - index_offset
#     filtered_df.loc[:, 'query_idx'] = filtered_df['query_idx'] - index_offset
#     return filtered_df

# # Filter the DataFrame for campus_train0 and campus_train1
# campus_train0_df = filter_pairs_by_range(pairs_df, train0_range)
# campus_train1_df = filter_pairs_by_range(pairs_df, train1_range, train1_range[0])

# # Save the splits into separate CSV files
# campus_train0_df.to_csv(train0_output, index=False)
# campus_train1_df.to_csv(train1_output, index=False)

# print(f"campus_train0 pairs saved to {train0_output}")
# print(f"campus_train1 pairs saved to {train1_output}")

campus_train0 pairs saved to /home/bjangley/VPR/mast3r-v2/pairs/ciampino_train0_pairs.csv
campus_train1 pairs saved to /home/bjangley/VPR/mast3r-v2/pairs/ciampino_train1_pairs.csv


In [10]:
import pandas as pd
import os
import random

# Input file and parameters
pairs_file = "/home/bjangley/VPR/mast3r-v2/pairs/ciampino_train1_pairs.csv"
pairs_df = pd.read_csv(pairs_file)
min_inliers = 300

# Filter rows based on the minimum number of inliers
filtered_rows = pairs_df[pairs_df['num_inliers'] > min_inliers]

# Calculate the number of pairs lost due to filtering
print(f"Total pairs before filtering: {len(pairs_df)}")
print(f"Total pairs after filtering: {len(filtered_rows)}")

# Extract relevant columns
anchor_idxs = filtered_rows['anchor_idx'].astype(int).tolist()
query_idxs = filtered_rows['query_idx'].astype(int).tolist()

# Combine anchor-query pairs into a list
# pairs = [(anchor_seqs[idx], anchor_idxs[idx], query_seqs[idx], query_idxs[idx]) for idx in range(len(anchor_idxs))]
pairs = [(anchor_idxs[idx], query_idxs[idx]) for idx in range(len(anchor_idxs))]

# Shuffle the pairs randomly
random.seed(42)  # Set seed for reproducibility
random.shuffle(pairs)

# Define split proportions
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# Calculate split indices
total_pairs = len(pairs)
train_end = int(total_pairs * train_ratio)
val_end = train_end + int(total_pairs * val_ratio)

# Split the pairs
train_pairs = pairs[:train_end]
val_pairs = pairs[train_end:val_end]
test_pairs = pairs[val_end:]
print(len(train_pairs), len(val_pairs), len(val_pairs))

Total pairs before filtering: 945
Total pairs after filtering: 832
582 124 124


In [11]:
# Output file paths
scene = "ciampino_train1"
pairs_path = f"/home/bjangley/VPR/mast3r-v2/pairs_finetuning/{scene}"

# Ensure the output directory exists
if not os.path.exists(pairs_path):
    os.makedirs(pairs_path)

train_output = os.path.join(pairs_path, f"train_pairs.txt")
val_output = os.path.join(pairs_path, f"val_pairs.txt")
test_output = os.path.join(pairs_path, f"test_pairs.txt")
all_output = os.path.join(pairs_path, f"all_pairs.txt")
# Save the splits into separate files
def save_pairs_to_file(pairs, output_file):
    with open(output_file, "w") as f:
        for pair in pairs:
            # Write anchor and query pair in the format: anchor_seq, anchor_idx, query_seq, query_idx
            f.write(f"{pair[0]} {pair[1]}\n")

save_pairs_to_file(train_pairs, train_output)
save_pairs_to_file(val_pairs, val_output)
save_pairs_to_file(test_pairs, test_output)
save_pairs_to_file(pairs, all_output)


In [ ]:
print(len(query_idxs))
print(len(anchor_idxs))


print("train", train_pairs[0:10])

In [12]:
from my_vbr_utils.vbr_dataset import VBRDataset, load_scene_calibration
from my_vbr_utils.utilities import load_scene_correspondences

# --- Config and Dataset Loading ---
# Set these variables to configure your scene and utility path
scene_name = 'ciampino_train1'
vbr_utils_root = '/home/bjangley/VPR/mast3r-v2/my_vbr_utils'

config_path = f'{vbr_utils_root}/vbrPaths.yaml'

# Load dataset and calibration
all_loaded = VBRDataset(config_path, scenes=[scene_name])
vbr_scene = all_loaded.get_combined_dataset()
calib = load_scene_calibration("ciampino", config_path)

Loading scene: ciampino_train1
  KITTI path: /datasets/vbr_slam/ciampino/ciampino_train1_kitti
  GT path: /datasets/vbr_slam/ciampino/ciampino_train1/ciampino_train1_gt.txt
  ✓ Successfully loaded ciampino_train1
Loaded 1 training scenes: ['ciampino_train1']


## Save Depth Maps

In [13]:
from PIL import Image
from my_utils.my_vbr_dataset import generate_depth_and_scene_maps
import numpy as np
OUTPUT_DIR="/home/bjangley/VPR/vbr/depths/ciampino_train1"
T_cam_lidar = calib['cam_l']["T_cam_lidar"]
K = calib['cam_l']['K']
# print(T_cam_lidar, K)
for idx in range(len(pairs)):
    pair = pairs[idx]
    for image in pair:
        item = vbr_scene[image]
        img_path = item['image']
        lidar_pts = item['lidar_points']
        if lidar_pts.shape[0] < 5:
            print(f"[{image}] Skipped (no lidar)")
            continue
        # Load image to get size
        img = Image.open(img_path)
        img_shape = img.size[::-1]  # (H, W)
        # Generate depth and scene maps
        depth, scene = generate_depth_and_scene_maps(lidar_pts, K, T_cam_lidar, img_shape)
        # Save .npy
        out_path = os.path.join(OUTPUT_DIR, f"{image:010d}.npy")
        np.save(out_path, depth.astype(np.float32))
        # print(f"[{image}] Saved to {out_path}")

In [8]:

# Count the number of .npy files in the output directory
npy_files = [f for f in os.listdir(OUTPUT_DIR) if f.endswith('.npy')]
num_npy_files = len(npy_files)

# Extract unique anchors and queries from the pairs
anchors = set(pair[0] for pair in pairs)
queries = set(pair[1] for pair in pairs)

num_unique_anchors = len(anchors)
num_unique_queries = len(queries)

# Print the results
print(f"Number of .npy files in {OUTPUT_DIR}: {num_npy_files}")
print(f"Number of unique anchors: {num_unique_anchors}")
print(f"Number of unique queries: {num_unique_queries}")

Number of .npy files in /home/bjangley/VPR/vbr/depths/ciampino_train0: 839
Number of unique anchors: 479
Number of unique queries: 360


In [9]:
import os
from my_utils.transformations import pose_to_se3, se3_to_pose
OUTPUT_DIR="/home/bjangley/VPR/vbr/poses"
# Output file path
poses_output_file = os.path.join(OUTPUT_DIR, "ciampino_train0.txt")

# Open the file for writing
with open(poses_output_file, "w") as f:
    for idx in range(len(vbr_scene)): 
        # Get the pose for the current image
        pose = vbr_scene.get_pose(idx)
        # Transform the pose to the camera frame
        # pose_cam = se3_to_pose(T_cam_lidar @ pose_to_se3(pose))
        # Write the pose to the file
        f.write(f"{pose[0]:.6f} {pose[1]:.6f} {pose[2]:.6f} {pose[3]:.6f} {pose[4]:.6f} {pose[5]:.6f} {pose[6]:.6f}\n")

print(f"Ground truth poses saved to {poses_output_file}")
    

Ground truth poses saved to /home/bjangley/VPR/vbr/poses/ciampino_train0.txt


In [ ]:

pose = vbr_scene.get_pose(1)
pose = se3_to_pose(T_cam_lidar @ pose_to_se3(pose))
print(T_cam_lidar)
# Print the pose in a readable format with decimals
print("Pose (Readable Format):")
print(f"x: {pose[0]:.6f}, y: {pose[1]:.6f}, z: {pose[2]:.6f}, "
      f"qx: {pose[3]:.6f}, qy: {pose[4]:.6f}, qz: {pose[5]:.6f}, qw: {pose[6]:.6f}")